# 1. Importing libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Reading the data

In [2]:
# storing the path

PROJECT_DIR = r"C:\Users\DELL\Desktop\Programming\ml-project2-flight-price-prediction-using-aws-sagemaker"
DATA_DIR = "data"

In [ ]:
# function which will get name of file and then return the df

def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)
    

In [4]:
# getting the data

flights = get_data("flight_price")
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contain 10,683 rows and 11 columns
- columns `Route` and `Total_stops` have 1 missing value
- datatype of some features are not correct for the analysis

# 3. Preliminary Analysis

## 3.1 Checking data types

In [8]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

## 3.2 Checking dublicates

In [10]:
# checking for dublicates
flights.duplicated().sum()

np.int64(220)

In [ ]:
# analysing all dublicate rows
(
    flights
    .loc[flights.duplicated(keep=False)]  # we have passed keep=False to get all the dublicate rows
    .sort_values(by=flights.columns.tolist())  # converting all the columns to list and then performing sorting
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
1535,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


## 3.3 Observations

- `Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be of datetime
- `Duration` and `Total_Stops` should be of numeric type
- There are 220 dublicate rows

# 4. Detailed analysis

## Airline

In [17]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some values are inconsistant/inaccurate

In [19]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

## Date of journey

In [24]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [25]:
pd.to_datetime(flights.Date_of_Journey, format="%d/%m/%Y")

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

## Source

In [27]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

## Destination

In [28]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

## Route

- This feature is rebundent and not adding any extra information

## Dep_Time

In [30]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [32]:
# checking if all the time values are in HH:MM format
(
    flights
    .Dep_Time
    .str.match(r"^\d{2}:\d{2}$")
    .all()
)

np.True_

In [34]:
pd.to_datetime(flights.Dep_Time, format="%H:%M").dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

## Arival_Time

In [ ]:
(
    flights
    .Arrival_Time
    .loc[flights.Arrival_Time.str.contains(r"\d{2}:\d{2}\+\d+")]
)

Series([], Name: Arrival_Time, dtype: object)

## duration

In [42]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [44]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    # .unique()

)

6474    5m
Name: Duration, dtype: object

In [45]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The row 6474 has wrong duration value so we will drop it

In [53]:
# handling all duration values
from operator import mul


(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True) # converting to dataframe
    .set_axis(["hours", "minutes"], axis=1)
    .assign(
        hours=lambda df_: (
            df_
            .hours
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_: (
            df_
            .minutes
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [ ]:
# validating the above result with original Duration column

from operator import mul


(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True) # converting to dataframe
    .set_axis(["hours", "minutes"], axis=1)
    .assign(
        hours=lambda df_: (
            df_
            .hours
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_: (
            df_
            .minutes
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename("Duration_in_mins")
    .to_frame()
    .join(flights.Duration.drop(index=6474), how="left")
)

,Duration_in_mins,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


## Total steps

In [88]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)
    # .astype(int)
    # .pipe(lambda ser: pd.to_numeric(ser))
    .replace(np.nan,pd.NA)
    .astype("Int64")
    .unique()
)

<IntegerArray>
[0, 2, 1, 3, <NA>, 4]
Length: 6, dtype: Int64

## Additional_info

In [79]:
(
    flights
    .Additional_Info
    .str.title()
    .value_counts()
    # .unique()
)

Additional_Info
No Info                         8348
In-Flight Meal Not Included     1982
No Check-In Baggage Included     320
1 Long Layover                    19
Change Airports                    7
Business Class                     4
1 Short Layover                    1
Red-Eye Flight                     1
2 Long Layover                     1
Name: count, dtype: int64

# 5. Cleaning the data

In [92]:
def convert_duration_to_mins(duration_ser: pd.Series) -> pd.Series:
    """Convert duration strings to total minutes as integers.

    Args:
        duration_ser (pd.Series): Series containing duration strings (e.g., "2h 30m").

    Returns:
        pd.Series: Series containing total duration in minutes as integers.
    """
    duration_in_mins = (
        duration_ser
        .str.split(" ", expand=True)  # converting to dataframe
        .set_axis(["hours", "minutes"], axis=1)
        .assign(
            hours=lambda df_: (
                df_
                .hours
                .str.replace("h", "")
                .astype(int)
                .mul(60)
            ),
            minutes=lambda df_: (
                df_
                .minutes
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )
    return duration_in_mins

In [ ]:
def clean_data(df):
    return (
        df
        .drop(index = 6474)  # dropping the row with invalid Duration value
        .drop_duplicates()  # dropping dublicate rows
        .assign(**{
            col: df[col].str.strip() # stripping whitespace from string columns
            for col in df.select_dtypes(include="object").columns
        })
        .rename(columns = str.lower)
        .assign(
            airline = lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey, format="%d/%m/%Y"),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_: df_.duration.pipe(convert_duration_to_mins),
            total_stops = lambda df_: (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .replace(np.nan,pd.NA)
                .astype("Int64")
            ),
            additional_info = lambda df_: df_.additional_info.str.title()
            
        )
        .drop(columns = ["route"])
        
    )

In [93]:
flight_cleaned = clean_data(flights)
flight_cleaned

C:\Users\DELL\AppData\Local\Temp\ipykernel_7140\2872250458.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
C:\Users\DELL\AppData\Local\Temp\ipykernel_7140\2872250458.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0,No Info,12648


# 6. Split the data (train, test, validation)

In [94]:
X = flight_cleaned.drop(columns=["price"])
y = flight_cleaned["price"]

X_, X_test, y_, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_, y_, test_size=0.25, random_state=42
)

X_train.shape, X_val.shape, X_test.shape

((6408, 9), (2137, 9), (2137, 9))

# 7. Export the splitted data

In [95]:
def export_data(X,y, name):
    data = X.copy()
    data["price"] = y
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    data.to_csv(file_path, index=False)
    return pd.read_csv(file_path).head()  # returning head of exported data for validation

In [96]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-18,Banglore,Delhi,17:00:00,19:50:00,170,0,No Info,6961
1,Jet Airways,2019-03-24,Mumbai,Hyderabad,07:10:00,08:35:00,85,0,No Info,5678
2,Jet Airways,2019-06-06,Kolkata,Banglore,08:25:00,19:50:00,685,1,No Info,14571
3,Vistara,2019-03-06,Mumbai,Hyderabad,07:30:00,19:55:00,745,1,No Info,12395
4,Indigo,2019-05-18,Kolkata,Banglore,15:30:00,18:05:00,155,0,No Info,4804


In [97]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2,No Info,16655
1,Goair,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1,No Info,4959
2,Vistara,2019-05-09,Kolkata,Banglore,20:20:00,09:05:00,765,1,No Info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0,No Info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1,In-Flight Meal Not Included,12898


In [98]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-Flight Meal Not Included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-Flight Meal Not Included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No Info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-Flight Meal Not Included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No Info,12242
